In [1]:
import json
import pandas as pd
from json_ntv import Ntv, NtvList
from datetime import date, time, datetime, timezone, timedelta

In [79]:
# pas les 'json'
types = pd.DataFrame({'ntv_type':  ['uint64', 'float32', 'string', 'datetime', 'int32', 'int64', 'float64', 'array', 'boolean'], 
                      'name_type': [None, None, None, None, None, 'int64', 'float64', 'array', 'boolean'], 
                      'dtype':     ['uint64', 'float32', 'string', 'datetime64[ns]', 'int32', 'int64', 'float64', 'object', 'bool']})

def from_ntv(ntv_value):
    ''' convert ntv_value into the return object'''
    ntv = Ntv.obj(ntv_value)
    ntv_name = ntv.name
    ntv_type = ntv.type_str if ntv.type_str != 'json' else ''
    #ntv = NtvList(ntv.val)
    ntv = NtvList(ntv.obj_value())
    
    if ntv_type in  types['ntv_type'].values or ntv_type == '':
        if ntv_type == '':
            dtype = None
            name_type = ''
        else:
            dtype = types.set_index('ntv_type').loc[ntv_type]['dtype']
            name_type = types.set_index('ntv_type').loc[ntv_type]['name_type']
        pd_name = ntv_name + '::' + name_type if name_type else ntv_name
        pd_name = pd_name if pd_name else None
        ntv_obj = ntv.to_obj(encoded=True) # simpleval=True
        return pd.read_json(ntv_obj, dtype=dtype, typ='series').rename(pd_name)
    dtype = 'object'
    name_type = ntv_type
    pd_name = ntv_name+'::'+name_type
    ntv_obj = ntv.to_obj(format='obj', simpleval=True)
    return pd.Series(ntv_obj, name=pd_name, dtype=dtype)

def to_ntv(sr):
    ''' convert object into the NTV entity'''
    sr_name = sr.name if sr.name else ''
    ntv_name, name_type = Ntv.from_obj_name(sr_name)[:2]
    dtype = sr.dtype.name
    
    if not name_type:
        types_none = types.set_index('name_type').loc[None]
        if dtype in types_none.dtype.values:
            ntv_type = types_none.set_index('dtype').loc[dtype].ntv_type
        else:
            ntv_type = 'json'
        ntv_value = json.loads(sr.to_json(orient='records', date_format='iso', default_handler=str))
    else:
        ntv_type = name_type
        if dtype == 'object':
            ntv_value = sr.to_list()   
        else:
            ntv_value = json.loads(sr.to_json(orient='records', date_format='iso', default_handler=str))
    return NtvList(ntv_value, ntv_name, ntv_type).to_obj()

In [75]:
a =  {'test::int32': [1,2,3]}
#a = [{'a': 3, 'e':5}, {'a': 4, 'e':6}]
print(Ntv.obj(a).obj_value())
print([ntv.val for ntv in Ntv.obj(a).val])
print(NtvList(Ntv.obj(a).val).val)
print(from_ntv(a))
print(from_ntv(NtvList(Ntv.obj(a).val).val))
print(to_ntv(from_ntv(a)) == a)

[1, 2, 3]
[1, 2, 3]
["sT", "sT", "sT"]
0    1
1    2
2    3
Name: test, dtype: int32
0    1
1    2
2    3
dtype: int32
True


In [76]:
for a in [{'test::int32': [1,2,3]},
          {'test': [1,2,3]},
          [1.0, 2.1, 3.0],
          ['er', 'et', 'ez'],
          [True, False, True],
          {'::boolean': [True, False, True]},
          {'::array': [{'a': 3, 'e':5}, {'a': 4, 'e':6}]},
          {'::string': ['er', 'et', 'ez']},
          {'test::float32': [1.0, 2.5, 3.0]},
          {'::int64': [1,2,3]},
          {'::datetime': ["2021-12-31T23:00:00.000","2022-01-01T23:00:00.000"] }]:
    #print(from_ntv(a))
    print(to_ntv(from_ntv(a)) == a)
#print(from_ntv({'test::float32': [1.0, 2.5, 3.0]}))
#print(to_ntv(from_ntv({'test::float32': [1.0, 2.5, 3.0]})))



True
True
True
True
True
True
True
True
True
True
True


In [86]:
sr = pd.Series([1,2,3], dtype='float64')
print(sr)
print(sr.dtype.name)

0    1.0
1    2.0
2    3.0
dtype: float64
float64


In [95]:
ntv_value = {'::date': ['2022-01-01', '2022-01-02']}
ntv = Ntv.obj(ntv_value)
ntv_name = ntv.name
ntv_type = ntv.type_str if ntv.type_str != 'json' else ''
#ntv = NtvList(ntv.val)
ntv = NtvList(ntv.obj_value())
obj = ntv.to_obj(format='obj', simpleval=True)
print(obj)

['2022-01-01', '2022-01-02']


In [94]:
sr = pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}])
sr = pd.Series([1,2,3], dtype='int64')
sr = pd.Series([1,2,3], dtype='float64')
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], name='::date')
ntv = to_ntv(sr)

ntv = Ntv.obj(ntv)
ntv_name = ntv.name
ntv_type = ntv.type_str if ntv.type_str != 'json' else ''
print(ntv_name, ntv_type)
print(type(ntv_name), type(ntv_type))
sr_name = sr.name if sr.name else ''
ntv_name, name_type = Ntv.from_obj_name(sr_name)[:2]
dtype = sr.dtype.name
print(sr_name, ntv_name, name_type, dtype)
print(sr)
print(to_ntv(sr))
print(from_ntv(to_ntv(sr)))
print(sr.equals(from_ntv(to_ntv(sr))))
print(type(from_ntv(to_ntv(sr))[0]))

 date
<class 'str'> <class 'str'>
::date None date object
0    2022-01-01
1    2022-01-02
Name: ::date, dtype: object
{'::date': ['2022-01-01', '2022-01-02']}
0    2022-01-01
1    2022-01-02
Name: ::date, dtype: object
False
<class 'str'>


In [90]:
srs = [pd.Series([1,2,3], dtype='int64'),
       pd.Series([True, False, True], dtype='bool'),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[3,4], [5,6]]),  
       pd.Series([[3,4], {'a': 3, 'e':5}]),  
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns, UTC]'), #1
       pd.Series([1,2,3], dtype='int32'), 
       pd.Series([1,2,3], dtype='uint64', name='::uint64'),
       pd.Series([1,2,3], dtype='float64', name='::float64'),
       pd.Series([1,2,3], dtype='object'), #
       pd.Series([[3,4], [5,6]], name='::array'),  
       pd.Series(['az', 'er', 'cd'], dtype='string'), 
       pd.Series([None, None, None]), #
       pd.Series([date(2022, 1, 1), date(2022, 1, 2)], name='::date') #
]
for sr in srs:
    print(from_ntv(to_ntv(sr)).equals(sr))
    #print(sr.equals(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series')))


True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
False
True
True
False
False


In [ ]:
srs = [pd.Series([1,2,3], dtype='int64'),
       pd.Series([True, False, True], dtype='bool'),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[3,4], [5,6]]),  
       pd.Series([[3,4], {'a': 3, 'e':5}]),  
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns, UTC]'),
       pd.Series([1,2,3], dtype='int32'),
       pd.Series([1,2,3], dtype='UInt64'),
       pd.Series([1,2,3], dtype='float64'),
       pd.Series([1,2,3], dtype='object'),
       pd.Series([(3,4), (5,6)]),  
       pd.Series(['az', 'er', 'cd'], dtype='string'),
       pd.Series([None, None, None]),
       pd.Series([date(2022, 1, 1), date(2022, 1, 2)])
]
for sr in srs:
    print(sr.equals(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series')))


In [ ]:
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], dtype='datetime64[ns, CET]')
print(sr)
sr2 = pd.Series([datetime(2022, 1, 1, tzinfo=timezone.utc), datetime(2022, 1, 2, tzinfo=timezone(timedelta(hours=2)))])
print(sr2)
print(type(sr2[1]))
print(sr.to_json(orient='records', date_format='iso', default_handler=str))
print(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series'))
print(pd.read_json(sr.to_json(orient='records', default_handler=str), dtype='datetime64[ns, UTC]', typ='series'))


In [ ]:
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], dtype='datetime64[ns, CET]', name='test')
print(sr)
js = sr.to_json(orient='records', date_format='iso', default_handler=str)
sr2 = pd.read_json(js, dtype='datetime64[ns, UTC]', typ='series').rename('test2')
print(sr2)

## mapping des types
Type json implicite (pas de ::type):
- json(number) - float64, int64
- json(string) - object(string)
- json(bool) - bool
- json(null) - NaT

Type pandas spécifique (::type non maintenu sur pandas):
- datetime - datetime64[ns]
- intxx, uintxx - Intxx, UIntxx
- floatxx - floatxx
- string - string

Type pandas explicite (::type maintenu sur pandas):
- float64 - float64
- int64 - Int64
- array - object
- boolean - bool
- null - NaT

Type  externe (::type maintenu sur pandas)

A voir
- datetime64[ns, <tz>]
- category
- period[<freq>]
- Sparse
- Interval

## conversion 
Type json:
- -> read_json(sans dtype)
- <- to_json

Type pandas spécifique ou explicite:
- -> read_json(avec dtype)
- <- to_json

Type externe:
- -> Ntv.to_obj(format='obj') -> Series
- <- Ntv.from_obj() <- Series

In [ ]:
a = pd.read_json('[false,true,null]', typ='series', dtype='Bool' )
print(type(a[0]))
print(type(a[2]))
print(a[0], '\n')
print(a)